In [1]:
#export
import csv
import numpy as np  # http://www.numpy.org
import ast
from datetime import datetime
from math import log, floor, ceil
import random
import numpy as np

Modify the Utility class's methods. You can also add additional methods as required but don't change existing methods' arguments.

In [2]:
#export
class Utility(object):
    
    # This method computes entropy for information gain
    def entropy(self, class_y):
        # Input:            
        #   class_y         : list of class labels (0's and 1's)

        # TODO: Compute the entropy for a list of classes
        #
        # Example:
        #    entropy([0,0,0,1,1,1,1,1,1]) = 0.918 (rounded to three decimal places)

        entropy = 0
        ### Implement your code here
        #############################################
        class_y = np.array(class_y)
        (unique, counts) = np.unique(class_y, return_counts=True)
        p = np.array(counts)/len(class_y)
        entropy = round(np.sum(-1*p.T*np.log2(p)), 3)
        #############################################
        return entropy


    def partition_classes(self, X, y, split_attribute, split_val):
        # Inputs:
        #   X               : data containing all attributes
        #   y               : labels
        #   split_attribute : column index of the attribute to split on
        #   split_val       : a numerical value to divide the split_attribute

        # TODO: Partition the data(X) and labels(y) based on the split value - BINARY SPLIT.
        # 
        # Split_val should be a numerical value
        # For example, your split_val could be the mean of the values of split_attribute
        #
        # You can perform the partition in the following way
        # Numeric Split Attribute:
        #   Split the data X into two lists(X_left and X_right) where the first list has all
        #   the rows where the split attribute is less than or equal to the split value, and the 
        #   second list has all the rows where the split attribute is greater than the split 
        #   value. Also create two lists(y_left and y_right) with the corresponding y labels.

        '''
        Example:

        X = [[3, 10],                 y = [1,
             [1, 22],                      1,
             [2, 28],                      0,
             [5, 32],                      0,
             [4, 32]]                      1]

        Here, columns 0 and 1 represent numeric attributes.

        Consider the case where we call the function with split_attribute = 0 and split_val = 3 (mean of column 0)
        Then we divide X into two lists - X_left, where column 0 is <= 3  and X_right, where column 0 is > 3.


        X_left = [[3, 10],                 y_left = [1,
                  [1, 22],                           1,
                  [2, 28]]                           0]

        X_right = [[5, 32],                y_right = [0,
                   [4, 32]]                           1]

        ''' 

        X_left = []
        X_right = []

        y_left = []
        y_right = []
        ### Implement your code here
        #############################################
        X = np.array(X)
        y = np.array(y).astype(int)
        data = np.concatenate((X,y.reshape(-1, 1)), axis=1)
        left = data[data[:, split_attribute] <= split_val]
        left1 = np.hsplit(left, len(X[0])+1)
        right = data[data[:, split_attribute] > split_val]
        right1 = np.hsplit(right, len(X[0])+1)
        
        X_left = np.concatenate((left1[0:len(X[0])]), axis=1).tolist()
        X_right = np.concatenate((right1[0:len(X[0])]), axis=1).tolist()
        y_left = left1[-1].astype(int).tolist()
        y_right = right1[-1].astype(int).tolist()
   
        #############################################
        return (X_left, X_right, y_left, y_right)


    def information_gain(self, previous_y, current_y):
        # Inputs:
        #   previous_y: the distribution of original labels (0's and 1's)
        #   current_y:  the distribution of labels after splitting based on a particular
        #               split attribute and split value

        # TODO: Compute and return the information gain from partitioning the previous_y labels
        # into the current_y labels.
        # You will need to use the entropy function above to compute information gain
        # Reference: http://www.cs.cmu.edu/afs/cs.cmu.edu/academic/class/15381-s06/www/DTs.pdf

        """
        Example:

        previous_y = [0,0,0,1,1,1]
        current_y = [[0,0], [1,1,1,0]]

        info_gain = 0.45915
        """

        info_gain = 0
        ### Implement your code here
        #############################################
        entropy_list = []
        for i in range(len(current_y)):
            current_y[i] = np.array(current_y[i])
            (u, c) = np.unique(current_y[i], return_counts=True)
            p = len(current_y[i])/len(previous_y)
            entropy_list.append(p * self.entropy(current_y[i]))
        entropy_sum = np.sum(entropy_list)
        info_gain = self.entropy(previous_y) - entropy_sum
        
        #############################################
        return info_gain


    def best_split(self, X, y):
        # Inputs:
        #   X       : Data containing all attributes
        #   y       : labels
        #   TODO    : For each node find the best split criteria and return the split attribute, 
        #             spliting value along with  X_left, X_right, y_left, y_right (using partition_classes) 
        #             in the dictionary format {'split_attribute':split_attribute, 'split_val':split_val, 
        #             'X_left':X_left, 'X_right':X_right, 'y_left':y_left, 'y_right':y_right, 'info_gain':info_gain}
        '''

        Example: 

        X = [[3, 10],                 y = [1, 
             [1, 22],                      1, 
             [2, 28],                      0, 
             [5, 32],                      0, 
             [4, 32]]                      1] 

        Starting entropy: 0.971 

        Calculate information gain at splits: (In this example, we are testing all values in an 
        attribute as a potential split value, but you can experiment with different values in your implementation) 

        feature 0:  -->    split_val = 1  -->  info_gain = 0.17 
                           split_val = 2  -->  info_gain = 0.01997 
                           split_val = 3  -->  info_gain = 0.01997 
                           split_val = 4  -->  info_gain = 0.32 
                           split_val = 5  -->  info_gain = 0 
                           
                           best info_gain = 0.32, best split_val = 4 


        feature 1:  -->    split_val = 10  -->  info_gain = 0.17 
                           split_val = 22  -->  info_gain = 0.41997 
                           split_val = 28  -->  info_gain = 0.01997 
                           split_val = 32  -->  info_gain = 0 

                           best info_gain = 0.4199, best split_val = 22 

 
       best_split_feature: 1  
       best_split_val: 22  

       'X_left': [[3, 10], [1, 22]]  
       'X_right': [[2, 28],[5, 32], [4, 32]]  

       'y_left': [1, 1]  
       'y_right': [0, 0, 1] 
        '''
        
        split_attribute = 0
        split_val = 0
        X_left, X_right, y_left, y_right = [], [], [], []
        ### Implement your code here
        #############################################
        y_array = np.array(y)
        X_array = np.hsplit(np.array(X), len(X[0]))
        
        ig_val = 0 
        best_val = 0
        best_attr = 0
        for attr in range(0,len(X[0])):
            for val in X_array[attr]:
                X_l, X_r, y_l, y_r = self.partition_classes(X, y, attr, val)
                ig = self.information_gain(y, [y_l, y_r])
                if ig > ig_val:
                    ig_val = ig
                    best_val = val
                    best_attr = attr
                else:
                    pass

        X_left, X_right, y_left, y_right = self.partition_classes(X, y, best_attr, best_val)
        
        return {'split_attribute':best_attr, 'split_val':best_val, \
         'X_left':X_left, 'X_right':X_right, 'y_left':y_left, 'y_right':y_right, 'info_gain':ig_val}
        #############################################
        

### Define the classes 'DecisionTree' and 'RandomForest'

Please modify the 'DecisionTree' and 'RandomForest' classes below

In [3]:
#export
class DecisionTree(object):
    def __init__(self, max_depth):
        # Initializing the tree as an empty dictionary or list, as preferred
        self.tree = {}
        self.max_depth = max_depth
        
    	
    def learn(self, X, y, par_node = {}, depth=0):
        # TODO: Train the decision tree (self.tree) using the the sample X and labels y
        # You will have to make use of the functions in Utility class to train the tree
        
        # par_node is a parameter that is useful to pass additional information to call 
        # the learn method recursively. Its not mandatory to use this parameter

        # Use the function best_split in Utility class to get the best split and 
        # data corresponding to left and right child nodes
        
        # One possible way of implementing the tree:
        #    Each node in self.tree could be in the form of a dictionary:
        #       https://docs.python.org/2/library/stdtypes.html#mapping-types-dict
        #    For example, a non-leaf node with two children can have a 'left' key and  a 
        #    'right' key. You can add more keys which might help in classification
        #    (eg. split attribute and split value)
        ### Implement your code here
        #############################################

        if len(y) == 0:   # no data in this group
            return None
        elif all(x == y[0] for x in y):   # all y is the same in this group
            return {"label": y[0]}
        elif depth >= self.max_depth:   # max depth reached 
            return None
        else:
            u = Utility()
   
            X_left = u.best_split(X, y)["X_left"]
            X_right = u.best_split(X, y)["X_right"]
            y_left = u.best_split(X, y)["y_left"]
            y_right = u.best_split(X, y)["y_right"]
            
            par_node["attr"] = u.best_split(X, y)["split_attribute"]
            par_node["val"] = u.best_split(X, y)["split_val"]
            par_node["left"] = self.learn(X_left, y_left, {}, depth+1)   
            par_node["right"] = self.learn(X_right, y_right, {}, depth+1) 
            
            self.tree = par_node
            depth += 1
            
            return self.tree
        #############################################

    def classify(self, record):
        # TODO: classify the record using self.tree and return the predicted label
        ### Implement your code here
        #############################################
        record_array = np.array(record)
        labels = []
        the_tree = self.tree

        if type(record[0]) == int or type(record[0]) == float:
            while the_tree.get("val"):
                if record[the_tree["attr"]] <= the_tree["val"]:
                    the_tree = the_tree["left"]
                else:
                    the_tree = the_tree["right"]
            else:
                labels.append(the_tree.get("label"))    
            #return labels
        else:
            for i, c in enumerate(record_array):
                while the_tree.get("val"):
                    if c[the_tree["attr"]] <= the_tree["val"]:
                        the_tree = the_tree["left"]
                    else:
                        the_tree = the_tree["right"]
                else:
                    labels.append(the_tree.get("label"))    
        return labels
        
        
        #############################################

In [4]:
#export
# This starter code does not run. You will have to add your changes and
# turn in code that runs properly.

"""
Here, 
1. X is assumed to be a matrix with n rows and d columns where n is the
number of total records and d is the number of features of each record. 
2. y is assumed to be a vector of labels of length n.
3. XX is similar to X, except that XX also contains the data label for each
record.
"""

"""
This skeleton is provided to help you implement the assignment.You must 
implement the existing functions as necessary. You may add new functions
as long as they are called from within the given classes. 

VERY IMPORTANT!
Do NOT change the signature of the given functions.
Do NOT change any part of the main function APART from the forest_size parameter.  
"""


class RandomForest(object):
    num_trees = 0
    decision_trees = {}

    # the bootstrapping datasets for trees
    # bootstraps_datasets is a list of lists, where each list in bootstraps_datasets is a bootstrapped dataset.
    bootstraps_datasets = []

    # the true class labels, corresponding to records in the bootstrapping datasets
    # bootstraps_labels is a list of lists, where the 'i'th list contains the labels corresponding to records in
    # the 'i'th bootstrapped dataset.
    bootstraps_labels = []

    def __init__(self, num_trees):
        # Initialization done here
        self.num_trees = num_trees
        self.decision_trees = [DecisionTree(max_depth=10) for i in range(num_trees)]
        self.bootstraps_datasets = []
        self.bootstraps_labels = []
        
    def _bootstrapping(self, XX, n):
        # Reference: https://en.wikipedia.org/wiki/Bootstrapping_(statistics)
        #
        # TODO: Create a sample dataset of size n by sampling with replacement
        #       from the original dataset XX.
        # Note that you would also need to record the corresponding class labels
        # for the sampled records for training purposes.

        sample = [] # sampled dataset
        labels = []  # class labels for the sampled records
        ### Implement your code here
        #############################################
        XX = np.asarray(XX)
        xx_sample = np.stack(random.choices(XX, k=n))
        xx_sample1 = np.hsplit(xx_sample, len(xx_sample[0]))
        sample = np.concatenate((xx_sample1[0:-1]), axis=1)
        labels = xx_sample1[-1]

        #############################################
        return (sample, labels)

    def bootstrapping(self, XX):
        # Initializing the bootstrap datasets for each tree
        for i in range(self.num_trees):
            data_sample, data_label = self._bootstrapping(XX, len(XX))
            self.bootstraps_datasets.append(data_sample)
            self.bootstraps_labels.append(data_label)

    def fitting(self):
        # TODO: Train `num_trees` decision trees using the bootstraps datasets
        # and labels by calling the learn function from your DecisionTree class.
        ### Implement your code here
        #############################################
        for i in range(self.num_trees):
            self.decision_trees[i].learn(self.bootstraps_datasets[i], self.bootstraps_labels[i], {}, 0)
        #############################################

    def voting(self, X):
        y = []

        for record in X:
            # Following steps have been performed here:
            #   1. Find the set of trees that consider the record as an out-of-bag sample.
            #   2. Predict the label using each of the above found trees.
            #   3. Use majority vote to find the final label for this recod.
            votes = []
            
            for i in range(len(self.bootstraps_datasets)):
                dataset = self.bootstraps_datasets[i]
                
                if record not in dataset:
                    OOB_tree = self.decision_trees[i]
                    effective_vote = OOB_tree.classify(record)
                    votes.append(effective_vote)
                    
            v = np.array([item for sublist in votes for item in sublist]).astype(int).tolist()
            counts = np.bincount([item for sublist in v for item in sublist])
            #counts = np.bincount(votes)

            if len(counts) == 0:
                # TODO: Special case
                #  Handle the case where the record is not an out-of-bag sample
                #  for any of the trees.
                # NOTE - you can add few lines of codes above (but inside voting) to make this work
                ### Implement your code here
                #############################################
                d = self.decision_trees[0]
                vote = d.classify(record)
                y = np.append(y, vote)
                #############################################
            else:
                y = np.append(y, np.argmax(counts))
                
        return y

    def user(self):
        """
        :return: string
        your GTUsername, NOT your 9-Digit GTId  
        """
        ### Implement your code here
        #############################################
        return 'fyan40'
        #############################################

In [5]:
#export

# TODO: Determine the forest size according to your implementation. 
# This function will be used by the autograder to set your forest size during testing
# VERY IMPORTANT: Minimum forest_size should be 10
def get_forest_size():
    forest_size = 10
    return forest_size

# TODO: Determine random seed to set for reproducibility
# This function will be used by the autograder to set the random seed to obtain the same results you achieve locally
def get_random_seed():
    random_seed = 4
    return random_seed

### Do not modify the below cell
The cell below is provided to test that your random forest classifier can be successfully built and run. Similar steps will be used to build and run your code in Gradescope. Any additional testing of functions can be done in the cells below the `%run helpers/notebook2script submission` cell, as these will not be parsed by the autograder.

In [6]:
def run():
    np.random.seed(get_random_seed())
    # start time 
    start = datetime.now()
    X = list()
    y = list()
    XX = list()  # Contains data features and data labels
    numerical_cols = set([i for i in range(0, 9)])  # indices of numeric attributes (columns)

    # Loading data set
    print("reading the data")
    with open("pima-indians-diabetes.csv") as f:
        next(f, None)
        for line in csv.reader(f, delimiter=","):
            xline = []
            for i in range(len(line)):
                if i in numerical_cols:
                    xline.append(ast.literal_eval(line[i]))
                else:
                    xline.append(line[i])

            X.append(xline[:-1])
            y.append(xline[-1])
            XX.append(xline[:])

    # Initializing a random forest.
    randomForest = RandomForest(get_forest_size())

    # printing the name
    print("__Name: " + randomForest.user()+"__")

    # Creating the bootstrapping datasets
    print("creating the bootstrap datasets")
    randomForest.bootstrapping(XX)

    # Building trees in the forest
    print("fitting the forest")
    randomForest.fitting()

    # Calculating an unbiased error estimation of the random forest
    # based on out-of-bag (OOB) error estimate.
    y_predicted = randomForest.voting(X)

    # Comparing predicted and true labels
    results = [prediction == truth for prediction, truth in zip(y_predicted, y)]

    # Accuracy
    accuracy = float(results.count(True)) / float(len(results))

    print("accuracy: %.4f" % accuracy)
    print("OOB estimate: %.4f" % (1 - accuracy))

    # end time
    print("Execution time: " + str(datetime.now() - start))

In [7]:
 %run helpers/notebook2script submission

Late Policy:

      "I have read the late policy for CS6424."
    
Please type 'yes' to agree and continue>yes


Honor Pledge:

      "I have read the Collaboration and Academic Honesty policy for CS6424.
      I certify that I have or will use outside references only in accordance with
      this policy, that I have or will cite any such references via code comments,
      and that I have not or will not copy any portion of my submission from another
      past or current student."

    
Please type 'yes' to agree and continue>yes


Converted Q2.ipynb to submission\submission.py


In [8]:
# Call the run() function to test your implementation
# Use this cell and any cells below for additional testing
run()

reading the data
__Name: fyan40__
creating the bootstrap datasets
fitting the forest


KeyboardInterrupt: 